In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d

In [2]:
filename_in_dir = []

for root, dirs, files in os.walk('swing'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print(filename_in_dir)

['swing/fo_drive/EXER_20210109_101548.csv', 'swing/fo_drive/EXER_20210109_101412.csv', 'swing/fo_drive/EXER_20210109_101954.csv', 'swing/fo_drive/EXER_20210109_101453.csv', 'swing/fo_drive/EXER_20210107_152154.csv', 'swing/fo_drive/EXER_20210109_101952.csv', 'swing/fo_drive/EXER_20210109_101424.csv', 'swing/fo_drive/EXER_20210105_102123.csv', 'swing/fo_drive/EXER_20210109_101429.csv', 'swing/fo_drive/EXER_20210109_101943.csv', 'swing/fo_drive/EXER_20210109_095631.csv', 'swing/fo_drive/EXER_20210105_102110.csv', 'swing/fo_drive/EXER_20210105_102106.csv', 'swing/fo_drive/EXER_20210109_101706.csv', 'swing/fo_drive/EXER_20210109_145515.csv', 'swing/fo_drive/EXER_20210109_101847.csv', 'swing/fo_drive/EXER_20210105_102046.csv', 'swing/fo_drive/EXER_20210107_152238.csv', 'swing/fo_drive/EXER_20210109_101404.csv', 'swing/fo_drive/EXER_20210105_102147.csv', 'swing/fo_drive/EXER_20210105_101947.csv', 'swing/fo_drive/EXER_20210105_102200.csv', 'swing/fo_drive/EXER_20210105_102149.csv', 'swing/fo_

In [3]:
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)



In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

In [5]:
from sklearn.model_selection import train_test_split
total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)


In [6]:
nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))

In [7]:
##여기서부터 랜덤 포레스트 분류기를 사용해서 예측 -> 예측 정확도는 99%로 상당히 높은 수준##

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
import numpy as np
mode = 'SVM'
best_score = 0

for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        for kernel in ['linear','rbf','poly']:
            tmp_model = svm.SVC(kernel=kernel,gamma=gamma,C=C)
            scores = cross_val_score(tmp_model,d2_train_dataset,y_train,cv=10)
            score = np.mean(scores)
            
            if score>best_score:
                
                best_score = score
                best_parameter = {'kernel':kernel,'gamma':gamma,'C':C}
                print('best_parameter is change : ',best_parameter)
            else:
                print('remain :',best_parameter)
mod = svm.SVC(**best_parameter)

predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

rf_pred = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if rf_pred[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
best_parameter is change :  {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
remain : {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
remain : {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
best_parameter is change :  {'k